In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import keras
import tensorflow as tf
# import the necessary packages
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras import optimizers
from keras.optimizers import Adam
from keras.optimizers import Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
import matplotlib.pyplot as pyplot
from PIL import Image
import scipy.ndimage
from sklearn.preprocessing import StandardScaler

# **REGRESSION WITH AUTOENCODER**

In [3]:
import os
import pandas as pd

BASE_DIR = "../input/eye-diagram-1000"
train_folder = BASE_DIR+'/CNN_IL_Eye'
files_in_train = sorted(os.listdir(train_folder))
images=[i for i in files_in_train]
train_df = pd.DataFrame()
train_df['images']=[str(x) for x in images]
col_list=[3]
#normalization of data
train_df['labels']= pd.read_csv("../input/taginfo2/Tag_Info.csv",usecols=col_list)
#SS = StandardScaler()
#train_df['labels'].reshape(-1,1)
#sc_X=np.array(train_df['labels'])
#sc_X=sc_X.reshape((-1,1))
#train_df['labels']=SS.fit_transform(sc_X)
#print(train_df)

In [4]:
from tensorflow.keras import layers
#encoder
filters=(32, 64)
chanDim = -1
latentDim=16
# define the input to the encoder
inputs = layers.Input(shape=(256,256,1))
x = inputs
# loop over the number of filters
for f in filters:
	# apply a CONV => RELU => BN operation
	x = layers.Conv2D(f, (3, 3), strides=2, padding="same")(x)
	x = layers.LeakyReLU(alpha=0.2)(x)
	x = layers.BatchNormalization(axis=chanDim)(x)
# flatten the network and then construct our latent vector
volumeSize = K.int_shape(x)
x =layers.Flatten()(x)
x= layers.LeakyReLU(alpha=0.2)(x)
latent = layers.Dense(latentDim)(x)
# build the encoder model
encoder = Model(inputs, latent, name="encoder")

In [5]:
from tensorflow.keras.models import load_model
encoder= load_model('../input/encoder-weights-3layers-gray-64/encoder_eye_32_3_gray.h5')
x= encoder.output
y=tf.keras.layers.LeakyReLU(alpha=0.5)(x)
x=Dense(16)(y)
x=tf.keras.layers.LeakyReLU(alpha=0.4)(x)
x= Dense(1)(x)
#preds= tf.keras.activations.tanh(x)
preds=tf.keras.layers.LeakyReLU(alpha=0.3)(x)
model=Model(inputs=encoder.input,outputs=preds)

In [6]:
for layer in model.layers[:8]:
    layer.trainable=False
for layer in model.layers[8:]:
    layer.trainable=True

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 32)      320       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 64)        256   

In [ ]:
train_data_gen= ImageDataGenerator(rescale=1./255,validation_split=0.1)
train_generator = train_data_gen.flow_from_dataframe(dataframe=train_df,
                                                     directory="../input/eye-diagram-1000/CNN_IL_Eye",
                                                     x_col="images",
                                                     y_col="labels",
                                                     target_size=(256,256),
                                                     batch_size=10,
                                                     shuffle=False,
                                                     class_mode= 'raw',
                                                     color_mode="grayscale",
                                                     subset='training')
val_generator = train_data_gen.flow_from_dataframe(dataframe=train_df,
                                                   directory="../input/eye-diagram-1000/CNN_IL_Eye",
                                                   x_col="images",
                                                   y_col="labels",
                                                   target_size=(256,256),
                                                   batch_size=10,
                                                   shuffle=False,
                                                   color_mode="grayscale",
                                                   class_mode= 'raw', 
                                                   subset='validation')

In [ ]:
optimizer = Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
model.compile(optimizer=optimizer, loss='mse', metrics=['mape'])

In [ ]:
history_reg = model.fit(
       train_generator,
       epochs=20,
       validation_data=val_generator,
       verbose=1
    )

In [ ]:
pyplot.plot(history_reg.history['loss'], label='train')
pyplot.plot(history_reg.history['val_loss'], label='val')
pyplot.legend()
pyplot.savefig('plot_reg.png')
pyplot.show()